In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [2]:
df=pd.read_excel("titanic.xlsx")

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)

In [5]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [6]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [7]:
x_train,x_test,y_train,y_test=train_test_split(df.drop(columns=['Survived']),df["Survived"],test_size=0.2,random_state=42)

In [8]:
x_train.shape

(712, 7)

In [9]:
x_test.shape

(179, 7)

In [10]:
si_age=SimpleImputer()
si_embarked=SimpleImputer(strategy="most_frequent")

x_train_age= si_age.fit_transform(x_train[["Age"]])
x_train_embark=si_embarked.fit_transform(x_train[["Embarked"]])

x_test_age=si_age.transform(x_test[["Age"]])
x_test_embark=si_embarked.transform(x_test[["Embarked"]])


In [11]:
x_train.isnull().sum()

Pclass        0
Sex           0
Age         140
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [12]:
ohe_sex=OneHotEncoder(sparse=False,handle_unknown="ignore")
ohe_embarked=OneHotEncoder(sparse=False,handle_unknown="ignore")

x_train_sex=ohe_sex.fit_transform(x_train[["Age"]])
x_train_embark=ohe_embarked.fit_transform(x_train_embark)

x_test_sex=ohe_sex.transform(x_test[["Age"]])
x_test_embark=ohe_embarked.transform(x_test_embark)

In [13]:
x_train_rem=x_train.drop(columns=["Sex","Age","Embarked"])
x_test_rem=x_test.drop(columns=["Sex","Age","Embarked"])

In [14]:
x_train_rem.columns

Index(['Pclass', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [15]:
x_train_transform=np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embark),axis=1)
x_test_transform=np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embark),axis=1)

In [16]:
x_train_transform

array([[1., 0., 0., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 0., 1.],
       [3., 0., 0., ..., 0., 0., 1.],
       ...,
       [3., 2., 0., ..., 0., 0., 1.],
       [1., 1., 2., ..., 0., 0., 1.],
       [1., 0., 1., ..., 0., 0., 1.]])

In [17]:
clf=DecisionTreeClassifier()
clf.fit(x_train_transform,y_train)

DecisionTreeClassifier()

In [18]:
y_pred=clf.predict(x_test_transform)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7094972067039106

In [20]:
import pickle

In [23]:
pickle.dump(ohe_sex,open("ohe_sex.pkl","wb"))
pickle.dump(ohe_embarked,open("ohe_embarked.pkl","wb"))
pickle.dump(clf,open("clf.pkl","wb"))